In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
import os

pd.set_option('max_colwidth', 400)

In [2]:
# load csv file
cwd = os.getcwd()
complaints_info_df = pd.read_csv('../Resources/complaints_uploaded.csv')

# filter to only the columns that we are interested in

names = ['Date received','Product','Company','Date sent to company','Company response to consumer','Complaint ID']
new_df = complaints_info_df[names]

In [3]:
# rename the columns to developer friendly names
rename = {'Date received':'date_received' ,'Product':'product', 'Company':'company',
          'Date sent to company': 'date_sent_to_company','Company response to consumer': 'company_response_to_consumer',
          'Complaint ID': 'complaint_ID'}
cleaned_df = new_df.rename(columns = rename)

In [4]:
# get number of rows
cleaned_df.shape[0]

925182

In [5]:
# drop rows with missing data
cleaned_df = cleaned_df.dropna()

In [6]:
# get number of rows
cleaned_df.shape[0]

925182

In [7]:
# check all the data types for all the columns
cleaned_df.dtypes

date_received                   object
product                         object
company                         object
date_sent_to_company            object
company_response_to_consumer    object
complaint_ID                     int64
dtype: object

In [8]:
# chnage the to datetime objects
cleaned_df["date_received"] = pd.to_datetime(cleaned_df["date_received"])
cleaned_df["date_sent_to_company"] = pd.to_datetime(cleaned_df["date_sent_to_company"])
cleaned_df.dtypes

date_received                   datetime64[ns]
product                                 object
company                                 object
date_sent_to_company            datetime64[ns]
company_response_to_consumer            object
complaint_ID                             int64
dtype: object

In [9]:
# sort table by date_received
cleaned_df = cleaned_df.sort_values(by = ["date_received"], ignore_index = True)

In [10]:
cleaned_df.head(5)

,date_received,product,company,date_sent_to_company,company_response_to_consumer,complaint_ID
0,2022-04-01,Checking or savings account,WELLS FARGO & COMPANY,2022-04-01,Closed with explanation,5396097
1,2022-04-01,"Credit reporting, credit repair services, or other personal consumer reports","EQUIFAX, INC.",2022-04-01,Closed with explanation,5391803
2,2022-04-01,"Credit reporting, credit repair services, or other personal consumer reports","TRANSUNION INTERMEDIATE HOLDINGS, INC.",2022-04-01,Closed with non-monetary relief,5393784
3,2022-04-01,Debt collection,Resurgent Capital Services L.P.,2022-04-01,Closed with explanation,5393812
4,2022-04-01,"Credit reporting, credit repair services, or other personal consumer reports",CL Holdings LLC,2022-04-01,Closed with explanation,5395816


In [11]:
# check all distinct values of product column
cleaned_df["product"].unique()

array(['Checking or savings account',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Debt collection', 'Mortgage', 'Student loan',
       'Credit card or prepaid card',
       'Money transfer, virtual currency, or money service',
       'Payday loan, title loan, or personal loan',
       'Vehicle loan or lease'], dtype=object)

In [12]:
# number of companies
cleaned_df['company'].nunique()

3267

In [13]:
cleaned_df.shape[0]

925182

In [14]:
cleaned_df.to_csv('../Resources/cleaned.csv', encoding='utf-8', index=False)

In [15]:
# Create our database engine
engine = create_engine('sqlite:///../Resources/complaints.sqlite')

In [16]:
# Read data from CSV and load into a dataframe object
data = pd.read_csv('../Resources/cleaned.csv')
 
# Write data into the table in PostgreSQL database
data.to_sql('complaints',engine, if_exists = "append", index = False)

925182

In [17]:
# test to see if the data was uploaded
engine.execute('select * from complaints LIMIT 5').fetchall()

[('2022-04-01', 'Checking or savings account', 'WELLS FARGO & COMPANY', '2022-04-01', 'Closed with explanation', 5396097),
 ('2022-04-01', 'Credit reporting, credit repair services, or other personal consumer reports', 'EQUIFAX, INC.', '2022-04-01', 'Closed with explanation', 5391803),
 ('2022-04-01', 'Credit reporting, credit repair services, or other personal consumer reports', 'TRANSUNION INTERMEDIATE HOLDINGS, INC.', '2022-04-01', 'Closed with non-monetary relief', 5393784),
 ('2022-04-01', 'Debt collection', 'Resurgent Capital Services L.P.', '2022-04-01', 'Closed with explanation', 5393812),
 ('2022-04-01', 'Credit reporting, credit repair services, or other personal consumer reports', 'CL Holdings LLC', '2022-04-01', 'Closed with explanation', 5395816)]